In [ ]:
#Run this preamble to import some libraries that are available in google colab that are often useful.
#Numpy is good for efficiently working with array/vector/matrix data.
#Random is good for generating random numbers according to some (discrete or continuous) distribution
#Matplotlib is good for plotting
#Torch is PyTorch, which is the standard python library for creating and training ML models
#You may need to call other libraries for your code

import numpy as np
import random
import matplotlib.pyplot as plt
#import torch

from operator import neg  #for negating integers

#Generating lists of generators and relators for Coxeter and Artin groups

The following code should use Coxeter matrices encoded as numpy arrays to:

1. Generate lists of Coxeter group generators (as nonzero integers 1 to n) and Coxeter group relators
2. Generate lists of Artin group generators (as nonzero integers -n to n) and Artin group relators

## Coxeter group generation

Function to generate Coxeter group generators (gives a list of nonzero integers 1 through  𝑛 )

In [ ]:
def cox_gen(matrix):
  n = np.sum(matrix == 1)            #masks for 1s and sums all true values
  generators = list(range(1,n+1))    #generates integers from 1 through n
  return generators

Function to generate Coxeter group relators (currently gives a list of tuples; maybe reformat as array)

In [ ]:
def cox_rel(matrix):
  #generate pairs to check cases where s != s'
  generators = cox_gen(matrix)
  pairs = [(generators[i], generators[j]) for i in range(len(generators)) for j in range(i + 1, len(generators))]
  relators = []

  #getting relators of form s^2
  for g in generators:
    relators.append([g,g])

  #getting braid relators
  for p in pairs:
    m = matrix[p[0]-1,p[1]-1]       #subtracting 1 retrieves the correct row and column, eg what we call row 1 is actually indexed as row 0
    if np.any(np.isinf(m)):         #skipping to the next pair if m(s,s') = infinity
      continue
    relators.append(p*(int(m)))     #otherwise, appends the pair p m times, representing the relation (ss')^m(s,s') = e
  return relators

## Artin group generation

Function to generate Artin group generators and their inverses (gives a list of nonzero integers $-n$ to $n$)

In [ ]:
def artin_gen(matrix):
  n = np.sum(matrix == 1)            #masks for 1s and sums all true values
  generators = list(range(-n,n+1))   #generates integers from -n through n
  generators.remove(0)
  return generators

Function to generate Artin group relators (currently gives a list of lists)

In [ ]:
def artin_rel(matrix):
  #generate pairs to check cases where s != s'
  generators = cox_gen(matrix)
  pairs = [(generators[i], generators[j]) for i in range(len(generators)) for j in range(i + 1, len(generators))]

  relators = []

  #retrieving length m from m(s,s')
  for p in pairs:
    m = matrix[p[0]-1,p[1]-1]
    if np.any(np.isinf(m)):         #skipping to the next pair if m(s,s') = infinity
      continue

    #building pi(s,s',m)
    pi = []

    #alternating between s and s' for an m-length list
    for i in range(int(m)):
      if i % 2 == 0:
        pi.append(p[0])     #even indices give s
      else:
        pi.append(p[1])     #odd indices give s'

    #building pi(s',s, m) inverse
    pi_inv = []
    for i in range(int(m)):               #same process as above except
      if i % 2 != 0:
        pi_inv.append(p[0])               #even indices now give s'
      else:
        pi_inv.append(p[1])               #and odd indices give s
    pi_inv = list(map(neg, pi_inv))       #flip signs to denote inverses

    #combining pi and pi inverse
    relators.append(pi + pi_inv)

  return relators

#Subroutine A

In [ ]:
SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

def reduce_coxeter_word(w):
  reduced = True
  while reduced:
    reduced = False
    i = 0
    while i < len(w) - 1:
      if w[i] == w[i+1]:
        w = w[:i] + w[i+2:]
        i = max(i-1,0)
      else:
        i += 1
  return w

def coxeter_word_to_string(w):
  return "".join(f's{i}'.translate(SUB) for i in w)

In [ ]:
word = [0,2,2,4]
print("Before reducing: ", coxeter_word_to_string(word))
reduced = reduce_coxeter_word(word)
print("After being reduced: ",coxeter_word_to_string(reduced))

In [ ]:
SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

def reduce_artin_word(w):
  stack = []
  for x in w:
    if stack and stack[-1] == -x:
      stack.pop()
    else:
      stack.append(x)
  return stack

def artin_word_to_string(w):
  result = []
  for i in w:
    index = abs(i)
    gen = f's{str(index).translate(SUB)}'
    if i < 0:
      gen += '⁻¹'
    result.append(gen)
  return "".join(result)

In [ ]:
word = [0,-2,2,4]
print("Before reducing: ",artin_word_to_string(word))
reduced = reduce_artin_word(word)
print("After reducing: ",artin_word_to_string(reduced))